**1 Set up initial network**

In [1]:
import torch
from difflogic import LogicLayer, GroupSum
from chop import MaseGraph
from chop.passes import *


# setup basic network

model = torch.nn.Sequential(
    torch.nn.Flatten(),
    LogicLayer(4, 4),
    GroupSum(k=2, tau=20)
)

CUSTOM_OPS = {
    "modules": {
        LogicLayer: {
            "args": {
                "input": "data_in"
            },
            "toolchain": "INTERNAL_RTL",
            "module": "fixed_difflogic_logic",
            "dependence_files": [
                "difflogic_layers/rtl/fixed_difflogic_logic.sv",
                "difflogic_layers/rtl/fixed_difflogic_logic_neuron.sv",
            ],
        },
        GroupSum: {
            "args": {
                "input": "data_in"
            },
            "toolchain": "INTERNAL_RTL",
            "module": "fixed_difflogic_groupsum",
            "dependence_files": [
                "difflogic_layers/rtl/fixed_difflogic_groupsum.sv",
            ],
        },
    },
    "functions": {},
}

mg = MaseGraph(model, custom_ops=CUSTOM_OPS)
mg.draw("out-svg/dlg_init.svg")


# add input data and analyse

image = torch.randn((1,2,2))

mg, _ = init_metadata_analysis_pass(mg)
mg, _ = add_common_metadata_analysis_pass(mg, pass_args={"dummy_in": {"input_1": image} })
mg.draw("out-svg/dlg_post_common.svg")

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**2 Set up hardware passes**

In [2]:
from collections import OrderedDict

def difflogic_hardware_metadata_optimize_pass(graph, args={}):
    
    def _is_logiclayer(node):
        return node.meta["mase"]["common"]["mase_op"] == "user_defined_module"
    
    for node in graph.nodes:
        if _is_logiclayer(node):
            pre_common_args_md = node.meta["mase"]["common"]["args"]
            post_common_args_md = {}
            node.meta["mase"]["hardware"]["difflogic_args"] = {}
            for k, v in pre_common_args_md.items():
                if "data_in" not in k:
                    node.meta["mase"]["hardware"]["difflogic_args"][k] = v
                else:
                    post_common_args_md[k] = v
            post_common_args_md = OrderedDict(post_common_args_md)
            node.meta["mase"]["common"]["args"] = post_common_args_md
    return (graph, None)

In [3]:
from chop.passes.graph.analysis.add_metadata.add_hardware_metadata import *

def difflogic_hardware_force_fixed_flatten_pass(graph, args={}):
    for node in graph.nodes:
        if node.meta["mase"]["common"]["mase_op"] == "flatten":
            # add_component source
            node.meta["mase"]["hardware"]["toolchain"] = "INTERNAL_RTL"
            node.meta["mase"]["hardware"]["module"] = "fixed_difflogic_flatten"
            node.meta["mase"]["hardware"]["dependence_files"] = ["difflogic_layers/rtl/fixed_difflogic_flatten.sv"]
            # else
            add_verilog_param(node)
            add_extra_verilog_param(node, graph)
            graph.meta["mase"]["hardware"]["verilog_sources"] += node.meta["mase"]["hardware"]["dependence_files"]
    return (graph, None)

In [4]:
from collections import OrderedDict

def difflogic_annotate_io_nodes(graph, args={}):
    
    def _is_io_node(node):
        return node.op in ["placeholder", "output"]
    
    for node in graph.nodes:
        if _is_io_node(node):
            node.meta["mase"]["hardware"]["difflogic"] = True
    return (graph, None)

In [5]:
def print_metadata_pass(graph, args={}):
    if len(args) != 0:
        for node in graph.nodes:
            print(node.op)
            for k, v in node.meta["mase"][args["metadata_key"]].items():
                print(f"{k}: {v}")
            print()
    else:
        for node in graph.nodes:
            print(node.op)
            for k, v in node.meta["mase"]["common"].items():
                print(f"{k}: {v}")
            print()
    return mg, None

In [6]:
mg, _ = difflogic_hardware_metadata_optimize_pass(mg)

In [7]:
mg, _ = difflogic_annotate_io_nodes(mg)

In [8]:
mg, _ = add_hardware_metadata_analysis_pass(mg)

In [9]:
mg, _ = difflogic_hardware_force_fixed_flatten_pass(mg)

In [10]:
mg, _ = emit_verilog_top_transform_pass(mg)
mg, _ = emit_internal_rtl_transform_pass(mg)
mg, _ = emit_bram_transform_pass(mg)

In [11]:
print_metadata_pass(mg, {"metadata_key": "hardware"})

placeholder
is_implicit: True
difflogic: True
device_id: 0
max_parallelism: [4, 4, 4, 4]

call_module
is_implicit: False
device_id: -1
interface: {}
toolchain: INTERNAL_RTL
module: fixed_difflogic_flatten
dependence_files: ['difflogic_layers/rtl/fixed_difflogic_flatten.sv']
max_parallelism: [4, 4, 4, 4]
verilog_param: {'DATA_IN_0_TENSOR_SIZE_DIM_0': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_1': 2, 'DATA_OUT_0_TENSOR_SIZE_DIM_0': 4}

call_module
difflogic_args: {'weights': {'type': 'float', 'precision': [32], 'shape': [4, 16], 'from': None, 'value': Parameter containing:
tensor([[-1.1258, -1.1524, -0.2506, -0.4339,  0.8487,  0.6920, -0.3160, -2.1152,
          0.3223, -1.2633,  0.3500,  0.3081,  0.1198,  1.2377,  1.1168, -0.2473],
        [-1.3527, -1.6959,  0.5667,  0.7935,  0.5988, -1.5551, -0.3414,  1.8530,
          0.7502, -0.5855, -0.1734,  0.1835,  1.3894,  1.5863,  0.9463, -0.8437],
        [-0.6136,  0.0316, -0.4927,  0.2484,  0.4397,  0.1124,  0.6408,  0.4412,
         -0.1023,  0.7924, -

(<chop.ir.graph.mase_graph.MaseGraph at 0x7fff5dd35f90>, None)